In [1]:
print("Ok")

Ok


In [2]:
%pwd

'e:\\Projects\\Gen-Ai\\Projects\\Medical-Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'e:\\Projects\\Gen-Ai\\Projects\\Medical-Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [ ]:
extracted_data

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [ ]:
text_chunks

In [16]:
# from langchain.embeddings import HuggingFaceEmbeddings

# #Download the Embeddings from Hugging Face
# def download_hugging_face_embeddings():
#     embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#     return embeddings


In [19]:
from langchain_nomic import NomicEmbeddings

In [20]:
embeddings = NomicEmbeddings(
    model="nomic-embed-text-v1.5"
)

In [21]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 768


In [ ]:
# query_result

In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')
# OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [25]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=768, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [35]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [27]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [28]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [29]:
docsearch

In [30]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [31]:
retrieved_docs = retriever.invoke("What is Acne?")

In [32]:
retrieved_docs

[Document(id='a988c8a9-a75b-42cb-8a02-d84627e5af87', metadata={'page': 241.0, 'page_label': '242', 'source': 'Data\\Medical_book.pdf'}, page_content='KEY TERMS\nAcne—A skin condition in which raised bumps,\npimples, and cysts form on the face, neck, shoul-\nders and upper back.\nBacteria—Tiny, one-celled forms of life that cause\nmany diseases and infections.\nBowel—The intestine; a tube-like structure that\nextends from the stomach to the anus. Some diges-\ntive processes are carried out in the bowel before\nfood passes out of the body as waste.\nCyst—An abnormal sac or enclosed cavity in the\nbody, filled with liquid or partially solid material.'),
 Document(id='dd39015d-acce-4144-aa0e-577ca0fa9015', metadata={'page': 37.0, 'page_label': '38', 'source': 'Data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs

In [36]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.5,
    max_tokens=500,
)

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [38]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [39]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland in the brain, leading to increased growth in bone and soft tissue, as well as various other bodily disturbances. This condition is characterized by the excessive production of growth hormone, which can cause symptoms such as enlarged hands and feet, joint pain, and vision problems.


In [40]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I don't know.
